In [18]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from collections import defaultdict
import traceback
import json

ModuleNotFoundError: No module named 'defaultdict'

In [2]:
linux = True

In [3]:
# the source is here:https://docs.google.com/spreadsheets/d/1l8gf56ifdBwAptohozsdVidx9dUS-yNx2qxamYsooTw/edit?usp=sharing
if linux: # should move to s3
    file_path = '/home/ec2-user/data/datasets_feature_sets_v1-1.csv'
    df_feature_set = pd.read_csv(file_path)
else: 
    df_feature_set = pd.read_csv('/home/jovyan/projects/data/datasets_feature_sets_v1-1.csv')

In [4]:
df_feature_set[['dataset_id', 'dataset']] = df_feature_set[['dataset_id', 'dataset']].ffill()

In [5]:
df_feature_set.head()

,dataset_id,dataset,feature_set,status,current_prod _path,pheno-master-data-collection-v1-prod-us-east-1_relative_path,pheno-big-data-prod-us-east-1_relative_path,contains_sensitive,notes,count_parquet_relative_path,count_field_name,Unnamed: 11,Unnamed: 12,Seed_july,MBZUAI_july,UCSF_july,Cornell_july,Harvard- Joslin Diabetes Center,academic_f&f_july
0,0.0,population,population,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/popula...,/population/population.parquet,NaN,NaN,NaN,/population/population.parquet,sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1.0,events,events,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/events...,/events/events.parquet,NaN,NaN,NaN,/events/events.parquet,research_stage_date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2.0,anthropometrics,anthropometrics,READY on v1.1 Master bucket,s3://pheno-anthropometrics-derived-prod/v1.1/a...,/anthropometrics/anthropometrics.parquet,NaN,NaN,NaN,/anthropometrics/anthropometrics.parquet,bmi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,3.0,fundus,fundus,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/fun...,/fundus/fundus.parquet,NaN,NaN,NaN,/fundus/fundus.parquet,"fundus_image_left, fundus_image_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,3.0,fundus,microvasculature,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/mic...,/fundus/microvasculature.parquet,NaN,NaN,NaN,/fundus/microvasculature.parquet,"vessel_density_left, vessel_density_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [6]:
df_feature_set_ready = df_feature_set.query('status == "READY on v1.1 Master bucket"')
df_feature_set_ready.head()

,dataset_id,dataset,feature_set,status,current_prod _path,pheno-master-data-collection-v1-prod-us-east-1_relative_path,pheno-big-data-prod-us-east-1_relative_path,contains_sensitive,notes,count_parquet_relative_path,count_field_name,Unnamed: 11,Unnamed: 12,Seed_july,MBZUAI_july,UCSF_july,Cornell_july,Harvard- Joslin Diabetes Center,academic_f&f_july
0,0.0,population,population,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/popula...,/population/population.parquet,NaN,NaN,NaN,/population/population.parquet,sex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1.0,events,events,READY on v1.1 Master bucket,s3://pheno-population-derived-prod/v1.1/events...,/events/events.parquet,NaN,NaN,NaN,/events/events.parquet,research_stage_date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2.0,anthropometrics,anthropometrics,READY on v1.1 Master bucket,s3://pheno-anthropometrics-derived-prod/v1.1/a...,/anthropometrics/anthropometrics.parquet,NaN,NaN,NaN,/anthropometrics/anthropometrics.parquet,bmi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,3.0,fundus,fundus,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/fun...,/fundus/fundus.parquet,NaN,NaN,NaN,/fundus/fundus.parquet,"fundus_image_left, fundus_image_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,3.0,fundus,microvasculature,READY on v1.1 Master bucket,s3://pheno-retina-derived-prod/v1.1/fundus/mic...,/fundus/microvasculature.parquet,NaN,NaN,NaN,/fundus/microvasculature.parquet,"vessel_density_left, vessel_density_right",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [7]:
master_path = "s3://pheno-master-data-collection-v1-prod-us-east-1"

In [8]:
print(df_feature_set_ready.shape)
# none = [np.nan, pd.na]
df = df_feature_set_ready.query('@pd.notna(count_parquet_relative_path) and @pd.notna(count_field_name)') # in @none
df.shape

(64, 19)


(47, 19)

In [9]:
print(df.shape)
df_no_dup = df.drop_duplicates(['dataset', 'feature_set', 'count_parquet_relative_path', 'count_field_name'])
print(df_no_dup.shape)

(47, 19)
(47, 19)


In [10]:
# check for all the datasets that the count_parquet_relative_path and count_field_name exist
# count_dict = dict()
# for index, row in df_no_dup.iterrows():
#     full_path = f"{master_path}{row['count_parquet_relative_path']}"
#     dataset = row['dataset']

#     df_parquet = pd.read_parquet(full_path)
#     df_parquet = df_parquet.reset_index()
    
#     for col in row['count_field_name'].split(','):
#         # remove white space
#         col = col.strip()
#         if col not in df_parquet.columns:
#             print (f"ERROR: {col} not in {full_path}")
#     del df_parquet
 

In [11]:
# import pandas as pd
# df = pd.read_parquet('s3://pheno-master-data-collection-v1-prod-us-east-1/diet_logging/diet_logging_events.parquet')
# df.head()


In [12]:
count_dict = defaultdict(dict)
for index, row in df_no_dup.iterrows():
    try:
        if row['dataset'] == 'diet_logging':
            continue
        dataset = row['dataset']
        feature_set = row['feature_set']
        
        #read parquet file
        full_path = f"{master_path}{row['count_parquet_relative_path']}"
        df_parquet = pd.read_parquet(full_path)
        df_parquet = df_parquet.reset_index()
        
        res_dict =  {
            "df_parquet_shape": str(df_parquet.shape[0]),
            "count_field_name": row['count_field_name'],
            "count_parquet_relative_path": row['count_parquet_relative_path'],
            "particioant_id_nunique": str(df_parquet['participant_id'].nunique())
            }
        
        max_col = ''
        max_count = 0
        max_part = 0
        
        for col in row['count_field_name'].split(','):
            # remove white space
            col = col.strip()
            if col not in df_parquet.columns:
                print (f"ERROR: {col} not in {full_path} : {df_parquet.columns}")
                
            no_na_df = df_parquet.dropna(subset=[col])
            
            if dataset != 'population': 
                date_col = 'collection_date'
            if date_col not in no_na_df.columns:
                date_col = 'collection_timestamp'
            if date_col not in no_na_df.columns:
                print (f"ERROR: {date_col} not in {full_path} : {df_parquet.columns}")
                first_date = None
                last_date = None
            else:   
                first_date = no_na_df[date_col].min() # first date
                last_date = no_na_df[date_col].max() # last date
            
            nunique_participants_not_na = str(no_na_df.participant_id.nunique()) # number of unique participants
            count_col_notna = no_na_df.shape[0] # number of not na entries
            
            if no_na_df.shape[0] > max_count:
                max_count = no_na_df.shape[0]
                max_col = col
                
                
            res_dict[col] = {
                "count_col_notna": str(count_col_notna),  
                "nunique_participants_not_na" : str(nunique_participants_not_na),
                "first_date": str(first_date),
                "last_date": str(last_date),
            }
            
        res_dict['max_col'] = max_col
        count_dict[f"{int(row['dataset_id'])}-{dataset}"][feature_set] =
        count_dict[f"{int(row['dataset_id'])}-{row['dataset']}_{row['feature_set']}_{row['count_field_name']}"] = res_dict
        # write data to a file
        with open('temp_count_dict.json', 'w') as f:
            json.dump(count_dict, f)
        
    except Exception as e: 
        print(row['dataset'], full_path)
        print(traceback.format_exc())
    

with open('count_dict.json', 'w') as f:
            json.dump(count_dict, f)

population s3://pheno-master-data-collection-v1-prod-us-east-1/population/population.parquet
Traceback (most recent call last):
  File "/tmp/ipykernel_17373/2938924778.py", line 32, in <module>
    if date_col not in no_na_df.columns:
NameError: name 'date_col' is not defined

ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/events/events.parquet : Index(['participant_id', 'cohort', 'research_stage', 'array_index',
       'month_of_birth', 'year_of_birth', 'sex', 'research_stage_type',
       'visit_center', 'research_stage_timestamp', 'research_stage_date',
       'age_at_research_stage', 'study_id', 'timezone'],
      dtype='object')
ERROR: collection_timestamp not in s3://pheno-master-data-collection-v1-prod-us-east-1/fundus/microvasculature.parquet : Index(['participant_id', 'cohort', 'research_stage', 'array_index',
       'fractal_dimension_left', 'vessel_density_left', 'average_width_left',
       'distance_tortuosity_left', 'squared_curvatu

In [17]:
count_dict.keys()

dict_keys(['1.0_events_events_research_stage_date', '2.0_anthropometrics_anthropometrics_bmi', '3.0_fundus_fundus_fundus_image_left, fundus_image_right', '3.0_fundus_microvasculature_vessel_density_left, vessel_density_right', '3.0_fundus_images_fundus_image_left, fundus_image_right', '3.0_fundus_segmented_images_fundus_image_binary_artery_segmentation_left, fundus_image_binary_artery_segmentation_right', '4.0_liver_ultrasound_liver_ultrasound_liver_ultrasound_qbox_depth', '6.0_hand_grip_hand_grip_hand_grip_strength_left, hand_grip_strength_right', '7.0_blood_pressure_blood_pressure_lying_blood_pressure_diastolic', '8.0_serum_lipidomics_serum_lipidomics_collection_date', '9.0_sleep_sleep_total_sleep_time', '9.0_sleep_sleep_raw_total_sleep_time', '9.0_sleep_hrv_hrv_time_during_night,hrv_time_during_nrem,hrv_time_during_rem,hrv_time_during_wake', '9.0_sleep_hrv_raw_hrv_time_during_night,hrv_time_during_nrem,hrv_time_during_rem,hrv_time_during_wake', '9.0_sleep_channels_time_series_channe

In [ ]:
count_dict

{'0.0_population_population_sex': {'df_parquet_shape': '11179',
  'count_field_name': 'sex',
  'count_parquet_relative_path': '/population/population.parquet',
  'particioant_id_nunique': '11179',
  'sex': {'count_col_notna': '11179',
   'nunique_participants_not_na': '11179',
   'first_date': 'None',
   'last_date': 'None'},
  'max_col': 'sex'}}

In [16]:
df_t = pd.read_parquet('s3://pheno-master-data-collection-v1-prod-us-east-1/fundus/microvasculature.parquet')
df_t['fundus_image_binary_artery_segmentation_left'].head()

participant_id  cohort  research_stage  array_index
2578644992      10k     00_00_visit     0              s3://pheno-big-data-prod-us-east-1/fundus/10k/...
1321172995      10k     00_00_visit     0              s3://pheno-big-data-prod-us-east-1/fundus/10k/...
9810640900      10k     00_00_visit     0              s3://pheno-big-data-prod-us-east-1/fundus/10k/...
1802502158      10k     02_00_visit     0              s3://pheno-big-data-prod-us-east-1/fundus/10k/...
2283962386      10k     00_00_visit     0              s3://pheno-big-data-prod-us-east-1/fundus/10k/...
Name: fundus_image_binary_artery_segmentation_left, dtype: object

In [ ]:
diet_df = pd.read_parquet('s3://pheno-master-data-collection-v1-prod-us-east-1/diet_logging/raw_diet_logging_events.parquet')

: 

: 

In [ ]:
diet_df.head()